## Deliverables


In [ ]:
!pip3 install pd
!pip3 install scikit-learn nltk
import tarfile
import sklearn as sk
import nltk
import pandas as pd
import numpy as np
import os,sys
import math


from collections import Counter
with tarfile.open('mini_newsgroups.tar', 'r') as news:
    news.extractall()

df = pd.DataFrame(columns = ["docid", "directory", "filename", "content", "terms"])

for (root, dirs, files) in os.walk('mini_newsgroups', topdown=True):
    for name in files:
        try:
            filename = os.path.join(root, name)
            with open(filename, mode="r", encoding="ISO-8859-1") as f:
                content = f.read()

            next_index = len(df)
            df.loc[next_index] = [filename, root, name, content, None]

        except Exception as e:
            print("cannot open:", root, name, "due to error:", e)

if not df.empty:
    print(df.describe())
    total_docs = len(df)
    average_length = df['content'].apply(len).mean()
    word_counts = Counter(" ".join(df['content']).split())
    print(f"Number of documents: {total_docs}")
    print(f"Average doc length: {average_length} characters")
    df['content'] = df['content'].astype(str)
    df['content'] = df['content'].str.lower() 
    print("Missing values before handling: ")
    print(df['content'].isnull().sum())
    content = df.at[0, 'content']
    print(df.dtypes)
    print(content)

    print("FIXING")
    row = df.loc[df['docid'] == 'mini_newsgroups/talk.politics.mideast/76277']
    if not row.empty:
        content = row.iloc[0]['content']
        print(content)
    else:
        print("No matching row found for docid 'mini_newsgroups/talk.politics.mideast/76277'")
else: 
    print("EMPTY")



from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.datasets import load_svmlight_file

def evaluate(training_filename):
    feature_vectors, targets = load_svmlight_file(training_filename)
    clf = MultinomialNB() #Multinomial naive Bayes classifier 
    scores = cross_val_score(clf, feature_vectors, targets, cv=5, scoring='f1_macro')
    print("f1_macro: %0.2f (+/- %0.2f)" % (scores.mean(), 1.96*scores.std()/math.sqrt(5) ))




In [5]:
def extract_subject_body(content):
    lines = content.split("\n")
    subject = ""
    body = []

    # Find the subject
    for line in lines:
        if line.startswith("Subject:"):
            subject = line.replace("Subject:", "").strip()
            break

    # Find the body
    try:
        lines_index = [i for i, line in enumerate(lines) if line.startswith("Lines:")]
        if lines_index:
            # Try to extract the integer value after "Lines:"
            try:
                lines_count = int(lines[lines_index[0]].split(":")[1].strip())
                body = lines[-lines_count:]
            except ValueError:
                print(f"Unexpected value after 'Lines:' in content. Defaulting to all lines after 'Lines:'. Content: {lines[lines_index[0]]}. Filename: {filename}")
                body = lines[lines_index[0]+1:]
                ## df.drop(df[df['filename'] == filename].index, inplace=True)
    except Exception as e:
        print("Could not extract body from content.", e)
        

    return subject, " ".join(body)

## 1.1A Explanation
The above function ingests the content section of the newsgroup page and returns only the Subject and Body. 
The body is found by detemrining the number of lines in the body from the lines header item. If unable to find the lines item, 
we take all lines after the Lines header.

In [6]:
## 1.1
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
import re
import nltk
lemma = nltk.wordnet.WordNetLemmatizer()

with tarfile.open('mini_newsgroups.tar', 'r') as news:
    news.extractall()

def preprocessing(doc):
    # Improved preprocessing
    doc = re.sub(r'\S+@\S+', '', doc)
    doc = re.sub(r"[^a-zA-Z0-9\s]", " ", doc)  # This keeps only alphanumeric characters and spaces

    
    
    tokens = word_tokenize(doc)
    stop_words = set(stopwords.words('english'))
    tokens = [t.lower() for t in tokens if t.lower() not in stop_words]

    # Lemmatization -- 0.01 better performance then Stemming on TF-IDF
    lemma = nltk.wordnet.WordNetLemmatizer()
    lTerms = [lemma.lemmatize(t) for t in tokens]
    
    # # Stemming
    # stemmer = SnowballStemmer("english")
    # lTerms = [stemmer.stem(t) for t in tokens]

    # Remove numeric terms -- increased f1Macro by .02
    terms = [t for t in lTerms if t.isalpha()]

    

    return terms


# Extracting and preprocessing
df = pd.DataFrame(columns=["docid", "directory", "filename", "subject", "body", "content", "terms"])
for (root, dirs, files) in os.walk('mini_newsgroups', topdown=True):
    for name in files:
        try:
            filename = os.path.join(root, name)
            with open(filename, mode="r", encoding="ISO-8859-1") as f:
                content = f.read()
                subject, body = extract_subject_body(content)
                refined_content = subject + " " + body
                terms = preprocessing(refined_content)
                next_index = len(df)
                df.loc[next_index] = [filename, root, name, subject, body, refined_content, terms]
        except Exception as e:
            print("Cannot open:", root, name, "due to error:", e)

if not df.empty:
    content = df.at[0, 'terms']
    print(content)

    
else:
    print("EMPTY")

Unexpected value after 'Lines:' in content. Defaulting to all lines after 'Lines:'. Content: Lines: dog. Filename: mini_newsgroups/talk.politics.mideast/76277
['heil', 'hernlem', 'lebanese', 'resistance', 'force', 'detonated', 'bomb', 'israeli', 'occupation', 'patrol', 'lebanese', 'territory', 'two', 'day', 'ago', 'three', 'soldier', 'killed', 'two', 'wounded', 'retaliation', 'israeli', 'israeli', 'backed', 'force', 'wounded', 'civilian', 'bombarding', 'several', 'lebanese', 'village', 'ironically', 'israeli', 'government', 'justifies', 'occupation', 'lebanon', 'claiming', 'necessary', 'prevent', 'bombardment', 'israeli', 'village', 'congratulation', 'brave', 'men', 'lebanese', 'resistance', 'every', 'israeli', 'son', 'place', 'grave', 'underlining', 'moral', 'bankruptcy', 'israel', 'occupation', 'drawing', 'attention', 'israeli', 'government', 'policy', 'reckless', 'disregard', 'civilian', 'life', 'brad', 'hernlem', 'nice', 'three', 'people', 'murdered', 'bradly', 'overjoyed', 'hear',

## 1.1B Explanation 

Preprocessing: The Preprsssing funtion accepts the Subject and Body found in the extract_subject_body function and 
conducts a number of data cleaning and normalization actions. Firstly, we remove all non-alphanumeric values from the document.
Secondly, we tokenize each document using nltk.tokenize. Thirdly, we remobe the any stopwords from the aformentioned list of tokens.
I chose to use nltks stopwords as my default list. Next, we performed lemmatization using nltk.worldnet. We compared this to Stemming
where we found their to be a 0.01 incremental improvement when compared to any stemming operation.Finnaly, we removed all non numeric value 
from the list of terms which resulted in an additonal 0.02 point improvement.

In [7]:
#1.2

from collections import defaultdict

# Create a default dictionary to store document frequencies
doc_freq_counter = defaultdict(int)

# For each set of terms in a document, update the doc frequency counter
for terms_set in df['terms']:
    unique_terms_in_doc = set(terms_set)
    for term in unique_terms_in_doc:
        doc_freq_counter[term] += 1

# Convert the defaultdict to a list and then to a DataFrame
term_entries = [(term_id, term, doc_freq) for term_id, (term, doc_freq) in enumerate(doc_freq_counter.items())]

term_df = pd.DataFrame(term_entries, columns=['term_id', 'term', 'doc_frequency'])

# Sorting the DataFrame based on the term_id
term_df = term_df.sort_values(by='term_id').reset_index(drop=True)

# Size of the term dictionary
term_dict_size = len(term_df)

print(term_df.head())

# Most Frequent Term -- Top Decending Term
most_frequent_term_row = term_df.sort_values(by='doc_frequency', ascending=False).iloc[0]
most_frequent_term = most_frequent_term_row['term']
most_frequent_term_frequency = most_frequent_term_row['doc_frequency']

# Least Frequent Term
least_frequent_term_row = term_df.sort_values(by='doc_frequency',ascending=True ).iloc[0]
least_frequent_term = least_frequent_term_row['term']
least_frequent_term_frequency = least_frequent_term_row['doc_frequency']

print(f"Term DF:")
print(f"Size: {term_dict_size}")
print(f"Most Frequent Term: '{most_frequent_term}' with a document frequency of {most_frequent_term_frequency}.")
print(f"Least Frequent Term: '{least_frequent_term}' with a document frequency of {least_frequent_term_frequency}.")



   term_id       term  doc_frequency
0        0   claiming             31
1        1  overjoyed              1
2        2        son             26
3        3     policy             52
4        4       heil              4
Term DF:
Size: 27316
Most Frequent Term: 'one' with a document frequency of 802.
Least Frequent Term: 'kfjc' with a document frequency of 1.


## 1.2 Explanation

The above code analyzes the collection of documents and understand how often each unique term appears across all documents. 
A dictionary with a default value of 0 is utilized to keep a tally of each term's frequency.
This data is then organized into a DataFrame, where each term is assigned a unique identification 
number and the DataFrame is sorted accordingly. Additionally, the code calculates the total number of unique terms in 
the collection. Finally, the code identifies and displays the most and least frequently occurring terms in the documents. 

After completeing a few iterations of the term analysis process I realized that many of the most freqently listed terms were
nonalpha ie. - @, (, , > -- so returned preprocessing to remove these. Removing these terms improved f1 scores dramatically.


In [8]:
# 2.1

# Define the mapping dictionary
class_mapping = {
    'comp.graphics': 0, 'comp.os.ms-windows.misc': 0, 'comp.sys.ibm.pc.hardware': 0, 
    'comp.sys.mac.hardware': 0, 'comp.windows.x': 0,
    
    'rec.autos': 1, 'rec.motorcycles': 1, 'rec.sport.baseball': 1, 'rec.sport.hockey': 1,
    
    'sci.crypt': 2, 'sci.electronics': 2, 'sci.med': 2, 'sci.space': 2,
    
    'misc.forsale': 3,
    
    'talk.politics.misc': 4, 'talk.politics.guns': 4, 'talk.politics.mideast': 4,
    
    'talk.religion.misc': 5, 'alt.atheism': 5, 'soc.religion.christian': 5
}

# Use the mapping dictionary to assign class ID to each document
df['class_id'] = df['directory'].apply(lambda x: class_mapping.get(os.path.basename(x), -1))  # Extracting the last directory name for matching



samples = df.sample(n=20, random_state=1)

# TEST: Print out the newsgroup names and class IDs for each sample
# for index, row in samples.iterrows():
#     directory = row['directory']
#     class_id = row['class_id']
#     newsgroup_name = os.path.basename(directory)
#     print(f"Directory: {directory}")
#     print(f"Group: {newsgroup_name}")


# Check the head of the DataFrame to verify
print(df.head())




                                         docid  \
0  mini_newsgroups/talk.politics.mideast/75895   
1  mini_newsgroups/talk.politics.mideast/76277   
2  mini_newsgroups/talk.politics.mideast/76284   
3  mini_newsgroups/talk.politics.mideast/76080   
4  mini_newsgroups/talk.politics.mideast/76410   

                               directory filename  \
0  mini_newsgroups/talk.politics.mideast    75895   
1  mini_newsgroups/talk.politics.mideast    76277   
2  mini_newsgroups/talk.politics.mideast    76284   
3  mini_newsgroups/talk.politics.mideast    76080   
4  mini_newsgroups/talk.politics.mideast    76410   

                                             subject  \
0                                       Heil Hernlem   
1  Re: To be exact, 2.5 million readers enlighten...   
2  Accounts of Anti-Armenian Human Right Violatio...   
3  Re: American Jewish Congress Open Letter to Cl...   
4                               Re: Go Hizbollah II!   

                                           

## 2.1 Explantation

The code above is mapping newsgroup names to class IDs. It  assigns each document in the dataset a class ID based on the newsgroup
it belongs to, using the document's directory name.

In [9]:
# 2.2

total_docs = len(df)

import numpy as np
term_df['idf'] = np.log10(total_docs / term_df['doc_frequency'])
print(term_df.head)


# Create dictionaries for quick look-ups
term_to_id = dict(zip(term_df['term'], term_df['term_id']))
term_to_idf = dict(zip(term_df['term'], term_df['idf']))
print("OPEN")
with open('training.TF', 'w') as tf_file, open('training.TFIDF', 'w') as tfidf_file:
    for index, row in df.iterrows():
        # Calculate TF for the document
        tf = {}
        for term in row['terms']:
            tf[term] = tf.get(term, 0) + 1

        # Calculate TF for the document using term IDs
        tf_with_ids = {}
        for term, freq in tf.items():
            term_id = term_to_id.get(term)
            tf_with_ids[term_id] = freq

        # Write to the TF file in sorted order
        tf_sorted = sorted(tf_with_ids.items())
        tf_str = ' '.join([f"{k}:{v}" for k, v in tf_sorted])
        tf_file.write(f"{row['class_id']} {tf_str}\n")

        # Calculate TFIDF for the document using optimized look-ups
        tfidf = {}
        for term, freq in tf.items():
            term_id = term_to_id.get(term)
            idf = term_to_idf.get(term, 0)
            tfidf[term_id] = freq * idf

        # Write to the TFIDF file in sorted order
        tfidf_sorted = sorted(tfidf.items())
        tfidf_str = ' '.join([f"{k}:{v}" for k, v in tfidf_sorted])
        tfidf_file.write(f"{row['class_id']} {tfidf_str}\n")


# Display the contents of the training.TF file
# with open('training.TF', 'r') as f:
#     content_tf = f.read()
#     print(content_tf)

<bound method NDFrame.head of        term_id         term  doc_frequency       idf
0            0     claiming             31  1.809668
1            1    overjoyed              1  3.301030
2            2          son             26  1.886057
3            3       policy             52  1.585027
4            4         heil              4  2.698970
...        ...          ...            ...       ...
27311    27311         deaf              1  3.301030
27312    27312  foolishness              1  3.301030
27313    27313        soric              1  3.301030
27314    27314   ecological              1  3.301030
27315    27315     moltmann              1  3.301030

[27316 rows x 4 columns]>
OPEN


## 2.2 Explanation

The above code  calculates the two values for each document in the dataset - Term Frequency and Term Frequency-Inverse Document Frequency.
The Term Frequency is a count of how many times each term appears in a document, while the TF-IDF is a weighted measure 
that takes into account how frequent a term is across all documents. These values are then saved to two separate files,
'training.TF' and 'training.TFIDF'.

In [10]:

from sklearn.datasets import load_svmlight_file
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.datasets import load_svmlight_file

def evaluate(training_filename):
    feature_vectors, targets = load_svmlight_file(training_filename)
    clf = MultinomialNB() #Multinomial naive Bayes classifier 
    scores = cross_val_score(clf, feature_vectors, targets, cv=5, scoring='f1_macro')
    print("f1_macro: %0.2f (+/- %0.2f)" % (scores.mean(), 1.96 *scores.std() /math.sqrt(5) ))
    
print("Training.TF")
evaluate("/Users/samkempton/dataMining/training.TF")
print("Training.TFIDF")
evaluate("/Users/samkempton/dataMining/training.TFIDF")

Training.TF
f1_macro: 0.72 (+/- 0.04)
Training.TFIDF
f1_macro: 0.77 (+/- 0.03)


## Evaluation Explanation

The code above evaluates the Term Frequeny and TF-IDF files. It them prints the f1 scores for each set. 

From this process we find that our training.tf file results in a mean score of 0.72 with a standard deviation of 0.04.
We also find that our training.tfidf file results in a mean score of 0.77 with a standard deviation of 0.03.